# **##################### Problem 3 #####################**

In the following, you will load some data, i.e., simulation results from training a 500-neurons RNN to mimic target functions similar to calcium fluorescence signals collected from a population of active neurons in the prefrontal cortex of a mammalian nervous system. 

Your task is to analyze these data using the following a recipe, and plot the results.


As a first step, let's load and open the data. Please follow the instructions carefully.

1. Download the data file ('pset_p3.mat') from here:
https://drive.google.com/file/d/1imOA-ipWWrv-CXobZ2rMUiiUxN0Hsc8B/view?usp=sharing

2. Upload the dataset 'pset_p3.mat' to your personal Google Drive. Please just upload it to the top level, i.e. not within any folder.

3. Run the piece of code just below. 
The code connects to your Google Drive via this part here:
```
from google.colab import drive
drive.mount('/content/drive')
```
It will prompt you to open a new tab, sign in to your google account, and copy a password. You'll have to enter the password below the prompt. 

Note: if you correctly placed the file in the main folder of your Drive, you should not get any error. However, in case you get errors, you can try to look for the file manually: there's a Files button in the left column of this colab, where you can browse through your Google Drive. 


In [20]:
### Import libraries

import time
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Some more definitions for plotting

colors = np.array([[0.1254902 , 0.29019608, 0.52941176],
       [0.80784314, 0.36078431, 0.        ],
       [0.30588235, 0.60392157, 0.02352941],
       [0.64313725, 0.        , 0.        ]])

figsize = (8, 3)

### Import data file

from google.colab import drive
drive.mount('/content/drive')

data_dir = "drive/MyDrive/pset_p3.mat"
try:
    import hdf5storage
except:
    !pip install hdf5storage
    import hdf5storage
p3_info = hdf5storage.loadmat(data_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


If you manage to correctly load the file, then the file contains:
* Data in a neurons x time matrix used as target functions to train the above; 
* Initial random and trained recurrent interaction matrices (both NxN sized).  
* Firing rates of the trained RNN in a neurons x time matrix; 
* firing rates of the same RNN before training in a neurons x time matrix.

Specifically, the content of the file is as follows:

 ```
N     # N neurons
g     # variance of the initial weights

# Network activity of untrained and trained network
t     # time vector 
R0    # Rates of untrained network  (N x time)
R     # Rates of trained network  (N x time)

# Target activitiy and corresponding times
target_t            # time vector for targets
target_activity     # targets

J0    # Initial random weight matrix (NxN)
J     # Trained weight matrix (NxN)
```

In the following piece of code, we extract those quantities from the file.


In [21]:
#Load variables into name space

# Number of neurons, g
N = p3_info["N"]
g = p3_info["g"]
# Activity before and after training
t = p3_info["t"]
R0 = p3_info["R0"]
R = p3_info["R"]
# Training targets
target_t = p3_info["target_t"]
target_R = p3_info["target_activity"]
# Weight matrices
J0 = p3_info["J0"]
J = p3_info["J"]

# Join activity matrices
lbls = ["Before training", "After training", "Target"]
Rs = [R0, R, target_R]

## Dynamics

**a)** Plot the firing rate of 3 randomly selected units in the RNN super-imposed on their respective target functions. 


**SOLUTION:**

The target looks like a moving Gaussian bump, with higher neuron indices corresponding to later times. 

The trained activity tracks the target, even though in a noisy manner. 

For completeness, we visualize activity for all neurons in the target, initial and trained dataset.

In [22]:
# Plot activity for all neurons before/after training and target



**b)** You figured out how to do PCA in the first hands-on lab session. Can you generate the same state space diagram for the outputs of the trained network? Compared to the state space picture for the untrained RNN, how has training changed the picture? How about when compared to the “target data”?

**SOLUTION:** We use the scikit learn routine. For each dataset (R_i in Rs), we create a pca object, run the fit, and project the dataset on the principal components. 

In [23]:
# Compute PCA for all three matrices

# Number of components to keep
n_comp = 50



We now plot projections of activity of PCs.

In [26]:
# Plot activity in state space (projected separately on each datasets' PCs)

# 2D



It turns out that this doesn't reveal so much. All trajectories have somewhat circular elements, the untrained one also has some initial transient (the straight line). But we projected on different axes for each plot, so there's not so much to compare. 

Let's project all the trajectories on only one set of PCs, e.g. the one of the trained network, and plot this:

In [27]:
# Plot activity in state space (projected on trained dataset's PCs)

# Project all one the axes computed for the trained networks. 

# hint: use 

# pca_proj.transform(R_i.T)



This gives some more insight. First, the initial activity, despite being of similar magnitude, has only a small and very unstructured projection on the PCs of the trained network. But the trained network and the target evolve in a very similar manner -- the projection of the target is about the same size as when projected on its own PCs. (One could also quantify this by computing the projection of two sets of eigenvectors).

**c)** Can you compute, and plot, how many principal components of the RNN capture 90% of the total variance of the activity? This should be a graph plotting percent variance, between 0 and 100, on the y-axis as a function of PC # on the x-axis. 


**SOLUTION:**

In [ ]:
# Plot explained variance

fig = plt.figure(figsize=figsize)
ax = fig.subplots(1, 1)




The plot shows that the trained network (as well as the target) has a substantially reduced dimensionality compared with the untrained network. 10 PCs cover 90% of the variance. 

## Connectivity inferred from the RNN

**a)** How has training changed the recurrent interactions? One way to quantify that is to look at the overall magnitude of the change, i.e., the norm of the difference between the untrained random matrix and the trained matrix inferred from the RNN fit to data. Compare this to the norm of the difference between any other two random matrices with  same size and variance as the untrained matrix. 

**SOLUTION:** The norm has increased dramatically, almost 7-fold. 

In [ ]:
# Compare norm difference between untrained and trained weights

# Compare with norm diff of two random matrices


**b)** Can you make a histogram of the entries in the trained recurrent matrix and compare it to the histogram of the elements in the untrained, randomly initialized recurrent matrix? The y-axis should be the log-frequency of the binned elements, and the x-axis should be the binned weight values. 

**c)**  Can you replot the same histograms as in (b) so the y-axes reflect logarithmic probbility density?

**SOLUTION:**

In [ ]:
# Histogram of entries


The histograms reveal that the matrix changed dramatically: many weights are much larger in magnitude. However, the final distribution seems to be the sum of two distribution, where one is very similar to the distribution of untrained weights. 

To obtain a better idea what's going on there, it's worthwhile plotting the weight matrices themselves. 

In [ ]:
# Plot matrices



This looks strange! Only a small number of columns seem to be much larger than the rest. The reason is that actually only 50 units in the network were trained. For a deeper analysis of the network and its dynamics, it would probably make sense to separate these trained ones from the rest...

**d)** Have your code spit out the first 4 moments of the trained and random matrices, i.e., mean, variance, kurtosis, and skewness. Which of these modes has changed the most as a function of training to match the data, and what does it mean? 


In [ ]:
# First 4 moments of recurrent matrices
from scipy.stats import skew, kurtosis


The trained matrix is changed drastically on all moments. Relatively, the kurtosis changed most.

**e)** Now that you have some intuition for the spectral modes of interaction matrices, as in Problem 1 **e**, can you plot the eigenvalues of the trained matrix? How has the distribution changed relative to where it started, i.e., that of the eigenvalue spectrum of the randomly initialized matrix? What do you think the implications of this change are, at the level of the dynamics? 


**SOLUTION:**

In [ ]:
# Plot eigenvalue spectrum
fig = plt.figure(figsize=(10, 4))
axes = fig.subplots(1, 2)




The eigenvalues are drastically increased, and the distribution is less uniform. The latter points to dynamics that unfold in a lower-dimensional space, in agreement with the picture we obtained from PCA. 